In [1]:
from langchain_groq import ChatGroq

In [2]:
llm = ChatGroq(
    temperature=0, 
    groq_api_key= 'Keep Groq API Key Here...', 
    model_name="llama-3.3-70b-versatile"
)
response = llm.invoke("The first person to land on moon was ...")
print(response.content)

The first person to land on the moon was Neil Armstrong. He stepped out of the lunar module Eagle and onto the moon's surface on July 20, 1969, during the Apollo 11 mission. Armstrong famously declared, "That's one small step for man, one giant leap for mankind," as he became the first human to set foot on the moon.


In [3]:
from langchain_community.document_loaders import WebBaseLoader

# update url if page not found
loader = WebBaseLoader("https://jobs.nike.com/job/R-48983")
page_data = loader.load().pop().page_content
print(page_data)
# web scraping 

USER_AGENT environment variable not set, consider setting it to identify your requests.


Apply for Lead AI/ML Engineer

Search JobsSkip navigationSearch JobsNIKE, INC. JOBSContract JobsJoin The Talent CommunityLife @ NikeOverviewBenefitsBrandsOverviewJordanConverseTeamsOverviewAdministrative SupportAdvanced InnovationAir Manufacturing InnovationAviationCommunicationsCustomer ServiceDesignDigitalFacilitiesFinance & AccountingGovernment & Public AffairsHuman ResourcesInsights & AnalyticsLegalManufacturing & EngineeringMarketingMerchandisingPlanningPrivacyProcurementProduct Creation, Development & ManagementRetail CorporateRetail StoresSalesSocial & Community ImpactSports MarketingStrategic PlanningSupply Chain, Distribution & LogisticsSustainabilityTechnologyLocationsOverviewNike WHQNike New York HQEHQ: Hilversum, The NetherlandsELC: Laakdal, BelgiumGreater China HQDiversity, Equity & InclusionOverviewMilitary InclusionDisability InclusionIndigenous InclusionInternshipsData & AnalyticsLead AI/ML EngineerShanghai, China MainlandBecome a Part of the NIKE, Inc. Team
NIKE, Inc. 

In [4]:
from langchain_core.prompts import PromptTemplate

prompt_extract = PromptTemplate.from_template(
        """
        ### SCRAPED TEXT FROM WEBSITE:
        {page_data}
        ### INSTRUCTION:
        The scraped text is from the career's page of a website.
        Your job is to extract the job postings and return them in JSON format containing the 
        following keys: `role`, `experience`, `skills` and `description`.
        Only return the valid JSON.
        ### VALID JSON (NO PREAMBLE):    
        """
)

chain_extract = prompt_extract | llm 
# lang chain cahin using pipe operator '|' 
res = chain_extract.invoke(input={'page_data':page_data})
# print(res.content)
type(res.content)


str

In [5]:
print(res.content)

```json
{
  "role": "Lead AI/ML Engineer",
  "experience": "6+ years, 3+ years of demonstrated professional experience in AI/ML",
  "skills": [
    "Machine learning and deep learning frameworks like PyTorch, Tensorflow",
    "Big Data, real time and batch data processing",
    "Cloud platforms (e.g., AWS, GCP)",
    "Containerization and deployment of models to production",
    "Strong problem solving and analytical mindset",
    "Effective communication skills"
  ],
  "description": "Work with business teams to identify complex issues to solve with AI technology, build ML/AI prototypes, deploy AI powered applications, and monitor their business performances."
}
```


In [6]:
from langchain_core.output_parsers import JsonOutputParser

json_parser = JsonOutputParser()
json_res = json_parser.parse(res.content)
json_res
# convert to json

{'role': 'Lead AI/ML Engineer',
 'experience': '6+ years, 3+ years of demonstrated professional experience in AI/ML',
 'skills': ['Machine learning and deep learning frameworks like PyTorch, Tensorflow',
  'Big Data, real time and batch data processing',
  'Cloud platforms (e.g., AWS, GCP)',
  'Containerization and deployment of models to production',
  'Strong problem solving and analytical mindset',
  'Effective communication skills'],
 'description': 'Work with business teams to identify complex issues to solve with AI technology, build ML/AI prototypes, deploy AI powered applications, and monitor their business performances.'}

In [7]:
type(json_res)

dict

### Prepare Chromadb

In [8]:
import pandas as pd

df = pd.read_csv("my_portfolio.csv")
df
# this csv contains the portfolio links of each technology we offer in our company.


,Techstack,Links
0,"React, Node.js, MongoDB",https://example.com/react-portfolio
1,"Angular,.NET, SQL Server",https://example.com/angular-portfolio
2,"Vue.js, Ruby on Rails, PostgreSQL",https://example.com/vue-portfolio
3,"Python, Django, MySQL",https://example.com/python-portfolio
4,"Java, Spring Boot, Oracle",https://example.com/java-portfolio
5,"Flutter, Firebase, GraphQL",https://example.com/flutter-portfolio
6,"WordPress, PHP, MySQL",https://example.com/wordpress-portfolio
7,"Magento, PHP, MySQL",https://example.com/magento-portfolio
8,"React Native, Node.js, MongoDB",https://example.com/react-native-portfolio
9,"iOS, Swift, Core Data",https://example.com/ios-portfolio


- whenever a jobpositing chromadb should go through requements and pull the relevent portfolio for us..

In [ ]:
import uuid
import chromadb

client = chromadb.PersistentClient('vectorstore')
collection = client.get_or_create_collection(name="portfolio")

if not collection.count():
    for _, row in df.iterrows():
        collection.add(documents=row["Techstack"],
                       metadatas={"links": row["Links"]},
                       ids=[str(uuid.uuid4())])

# iterate through df and add it to chroma db
# metadat is links, Techstack is main doc.
# uuids are unique id generation perpose, it is a linux, url concept

In [17]:
links = collection.query(query_texts=['experiance in python skills', ' experiance in react'], n_results=2).get('metadatas', [])
links
# this gets python related portfolios from database

[[{'links': 'https://example.com/ml-python-portfolio'},
  {'links': 'https://example.com/python-portfolio'}],
 [{'links': 'https://example.com/react-portfolio'},
  {'links': 'https://example.com/react-native-portfolio'}]]

In [19]:
job = json_res
job['skills']

['Machine learning and deep learning frameworks like PyTorch, Tensorflow',
 'Productize machine learning models',
 'Big Data, real time and batch data processing',
 'Cloud platforms (e.g., AWS, GCP)',
 'Containerize and deploy models to production',
 'Strong problem solving and analytical mindset',
 'Effective communication skills']

In [21]:
job

{'role': 'Lead AI/ML Engineer',
 'experience': '6+ years, 3+ years of demonstrated professional experience in AI/ML',
 'skills': ['Machine learning and deep learning frameworks like PyTorch, Tensorflow',
  'Productize machine learning models',
  'Big Data, real time and batch data processing',
  'Cloud platforms (e.g., AWS, GCP)',
  'Containerize and deploy models to production',
  'Strong problem solving and analytical mindset',
  'Effective communication skills'],
 'description': 'Work with business teams to identify complex and challenging issues to solve with AI technology, build ML/AI prototypes, deploy AI powered applications, and monitor their business performances'}

In [20]:
links = collection.query(query_texts=job['skills'], n_results=2).get('metadatas', [])
links
# first initiate job below
# this matches my shils with job we scarapped

[[{'links': 'https://example.com/ml-python-portfolio'},
  {'links': 'https://example.com/python-portfolio'}],
 [{'links': 'https://example.com/ml-python-portfolio'},
  {'links': 'https://example.com/vue-portfolio'}],
 [{'links': 'https://example.com/ml-python-portfolio'},
  {'links': 'https://example.com/ios-portfolio'}],
 [{'links': 'https://example.com/xamarin-portfolio'},
  {'links': 'https://example.com/devops-portfolio'}],
 [{'links': 'https://example.com/devops-portfolio'},
  {'links': 'https://example.com/kotlin-backend-portfolio'}],
 [{'links': 'https://example.com/ml-python-portfolio'},
  {'links': 'https://example.com/angular-portfolio'}],
 [{'links': 'https://example.com/android-portfolio'},
  {'links': 'https://example.com/ml-python-portfolio'}]]

### Creating prompt for writing an email with all the data

In [ ]:
prompt_email = PromptTemplate.from_template(
        """
        ### JOB DESCRIPTION:
        {job_description}
        
        ### INSTRUCTION:
        You are Krish, a business development executive at IT Firm. IT Firm is an AI & Software Consulting company dedicated to facilitating
        the seamless integration of business processes through automated tools. 
        Over our experience, we have empowered numerous enterprises with tailored solutions, fostering scalability, 
        process optimization, cost reduction, and heightened overall efficiency. 
        Your job is to write a cold email to the client regarding the job mentioned above describing the capability of IT Firm 
        in fulfilling their needs.
        Also add the most relevant ones from the following links to showcase IT Firm's portfolio: {link_list}
        Remember you are Krish, BDE at IT Firm. 
        Do not provide a preamble.
        ### EMAIL (NO PREAMBLE):
        
        """
        )

chain_email = prompt_email | llm
res = chain_email.invoke({"job_description": str(job), "link_list": links})
print(res.content)